# BIGRU model met embeddings
## met embedding pre-training

In [ ]:
from torch import optim
from data.dataloader import DatasetLoader
from torch.utils.data import DataLoader
import torch
from models.BiGRU import BiGRU
from torch import nn
from tqdm import tqdm

## 1. Dataset uitlezen

In [ ]:
dataset = DatasetLoader("eventlog/phone_usage_cleaned.csv", seq_length=10, max_apps=200)
removeapps = ["Screen on (locked)",
              "Screen off (locked)",
              "Screen on (unlocked)",
              "Screen off",
              "Samsung Experience Service",
              "Package installer",
              "System UI",
              "Customisation Service",
              "Configuration update",
              "EmergencyManagerService",
              "DeviceKeystring",
              "Samsung Keyboard",
              "HwModuleTest",
              "Device shutdown",
              "Device boot"]
dataset.clean(removeapps)
train_dataloader = DataLoader(dataset)

## 2. bigru model aanmaken en trainen

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
bigru = BiGRU(max_apps=200,
              seq_length=10,
              n_gru=64)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(bigru.parameters(), lr=0.0015)
bigru.to(device=device)

running_loss = 0.0
i = 0
total = 0
correct = 0
epochs = 4
for prev_apps_indices, curr_app_index, curr_app_onehot in tqdm(train_dataloader):
    prev_apps_indices, curr_app_index = prev_apps_indices.to(device), curr_app_index.to(device)

    for j in range(epochs):
        optimizer.zero_grad()

        outputs = bigru(prev_apps_indices[0])

        loss = criterion(outputs, curr_app_index)
        loss.backward()
        optimizer.step()

        total += 1
        _, predicted = torch.topk(outputs.data, 5)
        correct += (curr_app_index in predicted)*1

        running_loss += loss.item()
    if i % 2000 == 1999:
        print('loss: %.3f' %
              (running_loss / 2000))
        running_loss = 0.0
        print('Accuracy of the network is:' + str(100 * correct / total))
        total = 0
        correct = 0
    i += 1
